# Classification_data_preprocessing (In Progress)

**Note: the code is still in progress and is subjected to change**

## Importing the required libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.svm import LinearSVC
from collections import Counter
from imblearn.base import BaseSampler
from keras import metrics
from sklearn import preprocessing
from sklearn.utils import resample

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## Reading the csv files (Datasets)

In [178]:
df_bath = pd.read_csv('C:/Users/osman/Desktop/diff_scene_annot/bath_annot/vott-csv-export/bath_data-export.csv')
df_bed = pd.read_csv('C:/Users/osman/Desktop/diff_scene_annot/bed_annot/vott-csv-export/bedroom-export.csv')
df_bowl = pd.read_csv('C:/Users/osman/Desktop/diff_scene_annot/bowl_annot/vott-csv-export/bowling-export.csv')
df_dine = pd.read_csv('C:/Users/osman/Desktop/diff_scene_annot/dine_annot/vott-csv-export/dine-export.csv')
df_kitchen = pd.read_csv('C:/Users/osman/Desktop/diff_scene_annot/Kitchen_annot/vott-csv-export/Object_Detector-export.csv')
df_office = pd.read_csv('C:/Users/osman/Desktop/diff_scene_annot/office_annot/vott-csv-export/office-export.csv')

In [179]:
df_bath['label'].unique()

array(['mirror', 'sink', 'toilet seat', 'bath tub', 'towel'], dtype=object)

In [180]:
df_bed['label'].unique()

array(['bed', 'dresser cabinet', 'night table', 'night lamp'],
      dtype=object)

In [181]:
df_bowl['label'].unique()

array(['Tv screen', 'Bowling rack', 'Bowling ball', 'bowling pins'],
      dtype=object)

In [182]:
df_dine['label'].unique()

array(['dining table', 'dining table lamp', 'vase', 'table lamp'],
      dtype=object)

In [183]:
df_kitchen['label'].unique()

array(['refrigerator', 'kitchen sink', 'oven', 'range hood'], dtype=object)

In [184]:
df_office['label'].unique()

array(['chair', 'printer', 'computer', 'Monitor', 'Desk', 'keyboard'],
      dtype=object)

In [186]:
data_class=pd.read_csv("class_dataset.csv", index_col=0)
data_pos=pd.read_csv("geo_pos_dataset.csv",index_col=0)
data_both=pd.read_csv("class_geo_dataset.csv", index_col=0)

In [167]:
data_chk=pd.read_csv("class_chk.csv", index_col=0)

In [177]:
data_chk['image'].value_counts().head(15)


b11.jpg                                2
b8.jpg                                 2
b3.jpg                                 2
b5.jpg                                 2
b7.jpg                                 2
b9.jpg                                 2
b10.jpg                                2
b12.jpg                                2
b4.jpg                                 2
b1.jpg                                 2
b2.jpg                                 2
img_0018.jpg                           2
b6.jpg                                 2
castell_casa_grande_ver_a_20000.jpg    1
4214_qto_casal_903.jpg                 1
Name: image, dtype: int64

In [187]:
data_class

,mirror,sink,toilet seat,bath tub,towel,bed,dresser cabinet,night table,table lamp,Tv screen,...,oven,range hood,chair,printer,computer,Monitor,Desk,keyboard,Class,image
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,bathroom,008.jpg
1,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,bathroom,100_0281.jpg
2,1,1,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,bathroom,100_1412.jpg
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,bathroom,14_master_bathroom_jl.jpg
4,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,bathroom,b1.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,2,office,office_10.jpg
565,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,2,0,2,office,office_13.jpg
566,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,office,or_02_03_0212_11_l.jpg
567,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,office,or_02_05_0479_11_l.jpg


In [188]:
df4 = data_class[ data_class['Class'] == 'bathroom' ]
df4['night table'].value_counts()
#for data in df4:
 #   data_class['chair']
        

0    97
Name: night table, dtype: int64

In [190]:

df4 = data_class[ data_class['Class'] == 'bed' ].index
for data in df4:
    data_class.loc[data,'Class']='bedroom'
    

In [191]:
data_class.Class.value_counts()

dining room       100
bedroom            98
bowling_lounge     98
bathroom           97
office             97
kitchen            79
Name: Class, dtype: int64

In [199]:
df4= data_class[data_class['Class']=='bedroom'].chair.value_counts()
df4

0    88
1    10
Name: chair, dtype: int64

In [193]:
df4 = data_class[ data_class['Class'] == 'bedroom' ].index
counter=0
for data in df4:
    if counter%10==0:
        data_class.loc[data,'chair']+=1
    counter+=1    

In [198]:
df4= data_class[data_class['Class']=='dining room'].chair.value_counts()
df4

0    80
1    15
2     5
Name: chair, dtype: int64

In [195]:
df4 = data_class[ data_class['Class'] == 'dining room' ].index
counter=0
for data in df4:
    if counter%20==0:
        data_class.loc[data,'chair']+=1
    counter+=1 

In [197]:
df4 = data_class[ data_class['Class'] == 'dining room' ].index
counter=0
for data in df4:
    if counter%5==0:
        data_class.loc[data,'chair']+=1
    counter+=1 

In [200]:
data_both

,mirror,sink,toilet seat,bath tub,towel,bed,dresser cabinet,night table,table lamp,Tv screen,...,val_91,val_92,val_93,val_94,val_95,val_96,val_97,val_98,val_99,Class
0,1,1,1,1,1,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,bathroom
1,1,1,0,0,1,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,bathroom
2,1,1,0,0,2,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,bathroom
3,1,1,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,bathroom
4,1,1,0,1,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,bathroom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0,0,0,0,0,0,0,0,0,0,...,381.016980,0.000000,0.000000,0.000000,0.000000,67.542383,368.135614,139.40678,402.033919,office
565,0,0,0,0,0,0,0,0,0,0,...,490.239557,0.000000,0.000000,0.000000,0.000000,614.717565,456.513013,800.00000,533.000000,office
566,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,office
567,0,0,0,0,0,0,0,0,0,0,...,0.000000,73.871182,150.418079,197.966102,256.000000,0.000000,0.000000,0.00000,0.000000,office


In [201]:
df4 = data_both[ data_both['Class'] == 'bed' ].index
for data in df4:
    data_both.loc[data,'Class']='bedroom'

In [202]:
data_both.Class.value_counts()

dining room       100
bedroom            98
bowling_lounge     98
bathroom           97
office             97
kitchen            79
Name: Class, dtype: int64

In [205]:
df4= data_both[data_both['Class']=='bedroom'].chair.value_counts()
df4

0    88
1    10
Name: chair, dtype: int64

In [204]:
df4 = data_both[ data_both['Class'] == 'bedroom' ].index
counter=0
for data in df4:
    if counter%10==0:
        data_both.loc[data,'chair']+=1
    counter+=1    

In [205]:
df4= data_both[data_both['Class']=='bedroom'].chair.value_counts()
df4

0    88
1    10
Name: chair, dtype: int64

In [206]:
df4= data_both[data_both['Class']=='dining room'].chair.value_counts()
df4

0    100
Name: chair, dtype: int64

In [207]:
df4 = data_both[ data_both['Class'] == 'dining room' ].index
counter=0
for data in df4:
    if counter%5==0:
        data_both.loc[data,'chair']+=1
    counter+=1 

In [208]:
df4 = data_both[ data_both['Class'] == 'dining room' ].index
counter=0
for data in df4:
    if counter%20==0:
        data_both.loc[data,'chair']+=1
    counter+=1 

In [209]:
df4= data_both[data_both['Class']=='dining room'].chair.value_counts()
df4

0    80
1    15
2     5
Name: chair, dtype: int64

### upsampling

In [210]:
upsampled_df = resample(data_class, replace=True, n_samples=3500, random_state=0)
upsampled_df

,mirror,sink,toilet seat,bath tub,towel,bed,dresser cabinet,night table,table lamp,Tv screen,...,oven,range hood,chair,printer,computer,Monitor,Desk,keyboard,Class,image
559,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,2,0,0,office,office5.jpg
192,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,bedroom,IMG_2430.jpg
359,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,dining room,dining10.jpg
9,0,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,bathroom,b3.jpg
277,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,bowling_lounge,bowling_0094.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,dining room,dining013.jpg
345,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,dining room,dining035.jpg
149,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,bedroom,bed170.jpg
22,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,bathroom,bath26.jpg


In [211]:
upsampled_df.index.value_counts()

411    14
515    13
423    13
203    13
510    13
       ..
18      1
222     1
224     1
308     1
567     1
Length: 568, dtype: int64

In [212]:
upsampled_df[upsampled_df.index==174]

,mirror,sink,toilet seat,bath tub,towel,bed,dresser cabinet,night table,table lamp,Tv screen,...,oven,range hood,chair,printer,computer,Monitor,Desk,keyboard,Class,image
174,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,bedroom,chambre_double.jpg
174,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,bedroom,chambre_double.jpg
174,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,bedroom,chambre_double.jpg
174,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,bedroom,chambre_double.jpg
174,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,bedroom,chambre_double.jpg
174,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,bedroom,chambre_double.jpg
174,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,bedroom,chambre_double.jpg
174,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,bedroom,chambre_double.jpg
174,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,bedroom,chambre_double.jpg
174,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,bedroom,chambre_double.jpg


In [213]:
upsampled_df=upsampled_df.reset_index(drop=True)

### splitting input and output columns

In [214]:
ips_1=IPS_1=upsampled_df.iloc[:,0:25]
ips_1

,mirror,sink,toilet seat,bath tub,towel,bed,dresser cabinet,night table,table lamp,Tv screen,...,vase,refrigerator,oven,range hood,chair,printer,computer,Monitor,Desk,keyboard
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,2,0,0
1,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3497,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3498,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [215]:
ops_1=OPS_1=upsampled_df.iloc[:,25:26]
ops_1

,Class
0,office
1,bedroom
2,dining room
3,bathroom
4,bowling_lounge
...,...
3495,dining room
3496,dining room
3497,bedroom
3498,bathroom


In [89]:
ips_2=IPS_2=data_both.iloc[:,0:125]
ips_2

,mirror,sink,toilet seat,bath tub,towel,bed,dresser cabinet,night table,table lamp,Tv screen,...,val_90,val_91,val_92,val_93,val_94,val_95,val_96,val_97,val_98,val_99
0,1,1,1,1,1,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1,1,1,0,0,1,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
2,1,1,0,0,2,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,1,1,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
4,1,1,0,1,0,1,0,1,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0,0,0,0,0,0,0,0,0,0,...,241.101695,381.016980,0.000000,0.000000,0.000000,0.000000,67.542383,368.135614,139.40678,402.033919
565,0,0,0,0,0,0,0,0,0,0,...,800.000000,490.239557,0.000000,0.000000,0.000000,0.000000,614.717565,456.513013,800.00000,533.000000
566,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
567,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,73.871182,150.418079,197.966102,256.000000,0.000000,0.000000,0.00000,0.000000


In [91]:
ops_2=OPS_2=data_both.iloc[:,125:126]
ops_2

,Class
0,bathroom
1,bathroom
2,bathroom
3,bathroom
4,bathroom
...,...
564,office
565,office
566,office
567,office


In [224]:
ops_1.iloc[1624]

Class    bedroom
Name: 1624, dtype: object

In [216]:
ips_1.dtypes

mirror               int64
sink                 int64
toilet seat          int64
bath tub             int64
towel                int64
bed                  int64
dresser cabinet      int64
night table          int64
table lamp           int64
Tv screen            int64
Bowling rack         int64
Bowling ball         int64
bowling pins         int64
dining table         int64
dining table lamp    int64
vase                 int64
refrigerator         int64
oven                 int64
range hood           int64
chair                int64
printer              int64
computer             int64
Monitor              int64
Desk                 int64
keyboard             int64
dtype: object

In [67]:
ips_2.dtypes

mirror           int64
sink             int64
toilet seat      int64
bath tub         int64
towel            int64
                ...   
val_95         float64
val_96         float64
val_97         float64
val_98         float64
val_99         float64
Length: 125, dtype: object

In [217]:
ops_1.dtypes

Class    object
dtype: object

In [70]:
ops_2.dtypes

Class    object
dtype: object

In [218]:
ip_train_1, ip_test_1, op_train_1, op_test_1 = train_test_split(ips_1,ops_1, test_size = 0.10, random_state = 10)

In [219]:
ip_train_1

,mirror,sink,toilet seat,bath tub,towel,bed,dresser cabinet,night table,table lamp,Tv screen,...,vase,refrigerator,oven,range hood,chair,printer,computer,Monitor,Desk,keyboard
1708,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2663,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
494,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1624,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1152,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3441,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1344,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
527,0,0,0,0,0,0,0,0,0,10,...,0,0,0,0,0,0,0,0,0,0
3197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [220]:
# create the Labelencoder object
le = preprocessing.LabelEncoder()
op_train_1['Class'] = le.fit_transform(op_train_1['Class'])
op_train_1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Class
1708,1
2663,3
494,3
1624,1
1152,1
...,...
3441,4
1344,5
527,2
3197,2


### SMOTE

In [225]:
op_train_1['Class'].value_counts()

3    568
2    565
1    543
0    535
5    516
4    423
Name: Class, dtype: int64

In [122]:
data_class.iloc[9]

mirror                      0
sink                        1
toilet seat                 1
bath tub                    1
towel                       1
bed                         1
dresser cabinet             0
night table                 1
table lamp                  0
Tv screen                   0
Bowling rack                0
Bowling ball                0
bowling pins                0
dining table                0
dining table lamp           0
vase                        0
refrigerator                0
oven                        0
range hood                  0
chair                       0
printer                     0
computer                    0
Monitor                     0
Desk                        0
keyboard                    0
Class                bathroom
Name: 9, dtype: object

In [226]:
sm = SMOTE(ratio='all', random_state = 6)
ip_train_1_prep, op_train_1_prep = sm.fit_sample(ip_train_1, op_train_1.values.ravel())
print(sorted(Counter(op_train_1_prep).items()))

[(0, 568), (1, 568), (2, 568), (3, 568), (4, 568), (5, 568)]


In [107]:
SMOTE()

SMOTE(k=None, k_neighbors=5, kind='regular', m=None, m_neighbors=10, n_jobs=1,
      out_step=0.5, random_state=None, ratio='auto', svm_estimator=None)

In [75]:
op_train_1=to_categorical(op_train_1)

In [76]:
op_train_1

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

In [81]:
len(op_train_1)

455

In [85]:
op_train_1[451]

array([0., 0., 0., 1., 0., 0.], dtype=float32)

# Rough work

### ADASYN

In [227]:
#ip_train_1_ad, op_train_1_ad = ADASYN().fit_sample(ip_train_1, op_train_1.values.ravel())
#print(sorted(Counter(op_train_1_ad).items()))

RuntimeError: Not any neigbours belong to the majority class. This case will induce a NaN case with a division by zero. ADASYN is not suited for this specific dataset. Use SMOTE instead.

In [48]:
op_train_chk.Class=pd.Categorical(op_train_1.Class)

In [49]:
op_train_1['CLass_code'] = op_train_chk.Class.cat.codes

AttributeError: 'Categorical' object has no attribute 'cat'

In [53]:
op_train_1['Class']

567            office
295       dining room
91           bathroom
1            bathroom
94           bathroom
            ...      
369       dining room
320       dining room
527            office
125               bed
265    bowling_lounge
Name: Class, Length: 455, dtype: object